# Shimadzu - EXI Log Parser

This notebook takes the output of the EXI Log export from a Shimadzu X-ray system and produces a .csv file that can be appended to the Reject Analysis and Dose Metric Dashboard database.

Let's start by importing the required libraries.

In [28]:
import pandas as pd
import os

The template for the Reject Analysis and Dose Metric Dashboard is as follows:

In [39]:
cols_list_final = ['Asset Number','DeviceID','Manufacturer','Model','Image Date','Image Time','Body Part','View','Exposure Index','KAP (uGy.m2)','kVp','Exposure (mAs)','Exposure time (ms)','Image Status','Reject Reason']

df_template = pd.DataFrame(columns=[*cols_list_final])
df_template

,Asset Number,DeviceID,Manufacturer,Model,Image Date,Image Time,Body Part,View,Exposure Index,KAP (uGy.m2),kVp,Exposure (mAs),Exposure time (ms),Image Status,Reject Reason


To clean up an export, you'll need to manually put in the filepath of the .csv file you want to clean up. 
<br>
You will also be asked for your asset number so that we can group systems in the Dashboard by facility.

In [40]:
print("What is the filepath for the .csv file you want to clean up?")
f = input()
## Test filepath "C:/Users/BernardM/JupyterNotebooks/RejectAnalysis/inputdata/ShimadzuMobile_Ex.csv"

print()

print("What is the asset number of the system?")
AssetNumber = input()


What is the filepath for the .csv file you want to clean up?


 C:/Users/BernardM/JupyterNotebooks/RejectAnalysis/inputdata/ShimadzuMobile_Ex.csv



What is the asset number of the system?


 123456


Let's do some clean-up on the .csv file:
- Define the separator. By default, this will be ";". However, this can be set to something different during export (| or , or -). Change the code below if your separator is something other than ";".

Shimadzu logs don't record any unique identifier for the system. The ModalityStationName defaults to LOCALHOST which is not useful for filtering. 
<br>
Let's give the system a unique identifier so we can filter it out.

In [41]:
print("Give the system a unique ID (e.g. unique system ID, serial number or room name):")
DeviceID = input()

Give the system a unique ID (e.g. unique system ID, serial number or room name):


 Shim_Dog


In [42]:
df = pd.read_csv(f, sep = ',')
df

,Content date,Content time,Accession number,Exposure mode,Protocol,Bodypart examined,Patient orientation,View position,Operator name,Reject date,...,EIt,DI,kV,mA,ms,mAs,SID,SensorSN,Workspace,SOD
0,2/20/2018,17:51:26,1XR18015469,Rad,Chest Lscape Supine Grid,CHEST,L\F,AP,Radiographer,NaN,...,100,-0.925788,90.0,320.0,10.0,3.20,2000,230009A7,35x43 Detector,NaN
1,3/08/2019,9:50:44,1XR19021383,Rad,Chest Lscape AP Grid,CHEST,L\F,AP,Radiographer,NaN,...,100,2.685097,110.0,125.0,80.0,10.00,2000,230009A7,35x43 Detector,NaN
2,5/09/2019,17:31:31,1XR19040152,Rad,Chest Lscape AP Grid,CHEST,L\F,AP,Radiographer,NaN,...,100,1.450104,110.0,250.0,22.0,5.50,2000,230009A7,35x43 Detector,NaN
3,5/09/2019,17:39:05,1XR19040238,Rad,Chest Lscape Supine Grid,CHEST,L\F,AP,Radiographer,NaN,...,100,3.096761,100.0,320.0,13.0,4.16,2000,230009A7,35x43 Detector,NaN
4,5/09/2019,20:05:52,1XR19040280,Rad,Chest Lscape Supine Grid,CHEST,L\F,AP,Radiographer,NaN,...,100,1.003154,100.0,320.0,6.0,1.92,2000,230009A7,35x43 Detector,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134,8/09/2019,10:56:54,1XR19069552,Rad,Chest Lscape AP Grid,CHEST,L\F,AP,Radiographer,NaN,...,100,0.556204,110.0,250.0,22.0,5.50,2000,230009A7,35x43 Detector,NaN
3135,8/09/2019,11:33:04,1XR19069606,Rad,Chest Lscape AP Grid,CHEST,L\F,AP,Radiographer,NaN,...,100,-1.502118,110.0,160.0,44.0,7.04,2000,230009A7,35x43 Detector,NaN
3136,8/09/2019,11:56:08,1XR19069612,Rad,Chest Lscape Supine Grid,CHEST,L\F,AP,Radiographer,NaN,...,100,1.320723,100.0,320.0,8.0,2.56,2000,230009A7,35x43 Detector,NaN
3137,8/09/2019,11:57:48,1XR19069612,Rad,Chest Lscape Supine Grid,CHEST,L\F,AP,Radiographer,NaN,...,100,2.038196,100.0,320.0,9.0,2.88,2000,230009A7,35x43 Detector,NaN


If the .csv file has been read in correctly above, you should see a table with all the values from the log. 
Carestream logs keep a unique system ID in the Machine ID column. We can use that later.
<br>
<br>
Let's add a few other identifiers for filtering purposes:
- Asset Number
- Manufacturer
- Model


In [43]:
cols_list = ['Asset Number','Manufacturer','Model']
df = df.reindex(columns=[*cols_list,*df.columns.tolist()])

df['Asset Number'] = AssetNumber
df['DeviceID'] = DeviceID
df['Manufacturer'] = "Shimadzu"
df['Model'] = "MobileDart"
df

,Asset Number,Manufacturer,Model,Content date,Content time,Accession number,Exposure mode,Protocol,Bodypart examined,Patient orientation,...,DI,kV,mA,ms,mAs,SID,SensorSN,Workspace,SOD,DeviceID
0,123456,Shimadzu,MobileDart,2/20/2018,17:51:26,1XR18015469,Rad,Chest Lscape Supine Grid,CHEST,L\F,...,-0.925788,90.0,320.0,10.0,3.20,2000,230009A7,35x43 Detector,NaN,Shim_Dog
1,123456,Shimadzu,MobileDart,3/08/2019,9:50:44,1XR19021383,Rad,Chest Lscape AP Grid,CHEST,L\F,...,2.685097,110.0,125.0,80.0,10.00,2000,230009A7,35x43 Detector,NaN,Shim_Dog
2,123456,Shimadzu,MobileDart,5/09/2019,17:31:31,1XR19040152,Rad,Chest Lscape AP Grid,CHEST,L\F,...,1.450104,110.0,250.0,22.0,5.50,2000,230009A7,35x43 Detector,NaN,Shim_Dog
3,123456,Shimadzu,MobileDart,5/09/2019,17:39:05,1XR19040238,Rad,Chest Lscape Supine Grid,CHEST,L\F,...,3.096761,100.0,320.0,13.0,4.16,2000,230009A7,35x43 Detector,NaN,Shim_Dog
4,123456,Shimadzu,MobileDart,5/09/2019,20:05:52,1XR19040280,Rad,Chest Lscape Supine Grid,CHEST,L\F,...,1.003154,100.0,320.0,6.0,1.92,2000,230009A7,35x43 Detector,NaN,Shim_Dog
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134,123456,Shimadzu,MobileDart,8/09/2019,10:56:54,1XR19069552,Rad,Chest Lscape AP Grid,CHEST,L\F,...,0.556204,110.0,250.0,22.0,5.50,2000,230009A7,35x43 Detector,NaN,Shim_Dog
3135,123456,Shimadzu,MobileDart,8/09/2019,11:33:04,1XR19069606,Rad,Chest Lscape AP Grid,CHEST,L\F,...,-1.502118,110.0,160.0,44.0,7.04,2000,230009A7,35x43 Detector,NaN,Shim_Dog
3136,123456,Shimadzu,MobileDart,8/09/2019,11:56:08,1XR19069612,Rad,Chest Lscape Supine Grid,CHEST,L\F,...,1.320723,100.0,320.0,8.0,2.56,2000,230009A7,35x43 Detector,NaN,Shim_Dog
3137,123456,Shimadzu,MobileDart,8/09/2019,11:57:48,1XR19069612,Rad,Chest Lscape Supine Grid,CHEST,L\F,...,2.038196,100.0,320.0,9.0,2.88,2000,230009A7,35x43 Detector,NaN,Shim_Dog


Let's match the .csv columns into the template:
- rename the columns of the original .csv file to match the template
- remove any columns we don't need
- rearrange the columns to match the template

In [44]:
df = df.rename(columns={"Bodypart examined": "Body Part",
                        "View position": "View",
                        "EI": "Exposure Index",
                        "Reject date": "Image Status",
                        "Reject reason": "Reject Reason",
                        "Content date": "Image Date",
                        "Content time": "Image Time",
                        "DAP": "KAP (uGy.m2)",
                        "kV": "kVp",
                        "mAs": "Exposure (mAs)",
                        "ms": "Exposure time (ms)",
                       })

df_out = df[[*cols_list_final]]

df_out

,Asset Number,DeviceID,Manufacturer,Model,Image Date,Image Time,Body Part,View,Exposure Index,KAP (uGy.m2),kVp,Exposure (mAs),Exposure time (ms),Image Status,Reject Reason
0,123456,Shim_Dog,Shimadzu,MobileDart,2/20/2018,17:51:26,CHEST,AP,80.801832,211.0,90.0,3.20,10.0,NaN,NaN
1,123456,Shim_Dog,Shimadzu,MobileDart,3/08/2019,9:50:44,CHEST,AP,185.570818,684.0,110.0,10.00,80.0,NaN,NaN
2,123456,Shim_Dog,Shimadzu,MobileDart,5/09/2019,17:31:31,CHEST,AP,139.640169,221.0,110.0,5.50,22.0,NaN,NaN
3,123456,Shim_Dog,Shimadzu,MobileDart,5/09/2019,17:39:05,CHEST,AP,204.021587,381.0,100.0,4.16,13.0,NaN,NaN
4,123456,Shim_Dog,Shimadzu,MobileDart,5/09/2019,20:05:52,CHEST,AP,125.983995,174.0,100.0,1.92,6.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134,123456,Shim_Dog,Shimadzu,MobileDart,8/09/2019,10:56:54,CHEST,AP,113.663333,163.0,110.0,5.50,22.0,NaN,NaN
3135,123456,Shim_Dog,Shimadzu,MobileDart,8/09/2019,11:33:04,CHEST,AP,70.760060,370.0,110.0,7.04,44.0,NaN,NaN
3136,123456,Shim_Dog,Shimadzu,MobileDart,8/09/2019,11:56:08,CHEST,AP,135.541517,172.0,100.0,2.56,8.0,NaN,NaN
3137,123456,Shim_Dog,Shimadzu,MobileDart,8/09/2019,11:57:48,CHEST,AP,159.889360,254.0,100.0,2.88,9.0,NaN,NaN


Finally, let's export the cleaned up .csv file into an output file. Finally, let's export the cleaned up .csv file into an output file. By default, this creates a new .csv file with the name "df_out". Change the code below to rename it to something unique with a timestamp if preferred.

In [35]:
df_out.to_csv(r'C:\Users\BernardM\JupyterNotebooks\RejectAnalysis\outputdata\df_out.csv',index = False, header = True)

This output file can now be appended to the Reject Analysis and Dose Metric Dashboard database.